In [1]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

model = LLM(
    "simplescaling/s1.1-32B",
    tensor_parallel_size=4,
)
tok = AutoTokenizer.from_pretrained("simplescaling/s1-32B")

stop_token_ids = tok("<|im_end|>")["input_ids"]

sampling_params = SamplingParams(
    max_tokens=32768,
    min_tokens=0,
    stop_token_ids=stop_token_ids,
)


INFO 02-13 19:57:03 config.py:1861] Downcasting torch.float32 to torch.float16.
INFO 02-13 19:57:08 config.py:350] This model supports multiple tasks: {'embedding', 'generate'}. Defaulting to 'generate'.
INFO 02-13 19:57:08 config.py:1020] Defaulting to use mp for distributed inference
INFO 02-13 19:57:08 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='simplescaling/s1.1-32B', speculative_config=None, tokenizer='simplescaling/s1.1-32B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=Observability

Loading safetensors checkpoint shards:   0% Completed | 0/29 [00:00<?, ?it/s]


(VllmWorkerProcess pid=1638412) INFO 02-13 19:59:54 model_runner.py:1077] Loading model weights took 15.3916 GB
INFO 02-13 19:59:54 model_runner.py:1077] Loading model weights took 15.3916 GB
(VllmWorkerProcess pid=1638410) INFO 02-13 19:59:55 model_runner.py:1077] Loading model weights took 15.3916 GB
(VllmWorkerProcess pid=1638409) INFO 02-13 19:59:55 model_runner.py:1077] Loading model weights took 15.3916 GB
(VllmWorkerProcess pid=1638410) (VllmWorkerProcess pid=1638412) (VllmWorkerProcess pid=1638409) INFO 02-13 20:00:04 worker.py:232] Memory profiling results: total_gpu_memory=47.43GiB initial_memory_usage=16.01GiB peak_torch_memory=17.92GiB memory_usage_post_profile=16.17GiB non_torch_memory=0.77GiB kv_cache_size=24.01GiB gpu_memory_utilization=0.90
INFO 02-13 20:00:04 worker.py:232] Memory profiling results: total_gpu_memory=47.43GiB initial_memory_usage=15.98GiB peak_torch_memory=17.92GiB memory_usage_post_profile=16.09GiB non_torch_memory=0.69GiB kv_cache_size=24.08GiB gpu_me

In [2]:
from datasets import load_dataset
dataset = load_dataset("Maxwell-Jia/AIME_2024")
prompt = dataset["train"][1]['Problem']

prompt = "<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n" + prompt + "<|im_end|>\n<|im_start|>assistant\n"
o = model.generate(prompt, sampling_params=sampling_params)
print(o[0].outputs[0].text)


Processed prompts: 100%|██████████| 1/1 [10:36<00:00, 636.48s/it, est. speed input: 0.33 toks/s, output: 29.22 toks/s]

think
Okay, so I have to solve this geometry problem where we have points O, A, and B on the coordinate plane. O is at the origin (0,0), A is at (1/2, 0), and B is at (0, √3/2). Then there's this family of segments F, which are all the unit-length segments from the x-axis (where point P is) to the y-axis (where point Q is) in the first quadrant. I need to find a point C on the line segment AB, not at A or B, such that this point C doesn't lie on any other segment in the family F except for AB itself. Then I have to compute OC squared, which equals p/q where p and q are coprime, and find p + q.

First, let me visualize this. Points O, A, and B form a right triangle, since A is on the x-axis and B is on the y-axis. The coordinates suggest that triangle OAB is a 30-60-90 triangle. Yes, because in a 30-60-90 triangle, the sides are in the ratio 1 : √3 : 2. Here, OA is 1/2, OB is √3/2, so the hypotenuse AB would be (1/2)/cos(60°) = 1/2 / (1/2) = 1. Yes, that checks out because the distance 

In [5]:
new_prompt_v1 = """
Let $O(0,0), A(\\tfrac{1}{\\sqrt{2}}, 0),$ and $B(0, \\tfrac{1}{\\sqrt{2}})$ be points in the coordinate plane. Let $\\mathcal{F}$ be the family of segments $\\overline{PQ}$ of unit length lying in the first quadrant with $P$ on the $x$-axis and $Q$ on the $y$-axis. There is a unique point $C$ on $\\overline{AB}$, distinct from $A$ and $B$, that does not belong to any segment from $\\mathcal{F}$ other than $\\overline{AB}$. Then $OC^2 = \\tfrac{p}{q}$, where $p$ and $q$ are relatively prime positive integers. Find $pq$."""

new_prompt_v1 = "<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n" + new_prompt_v1 + "<|im_end|>\n<|im_start|>assistant\n"

o = model.generate(new_prompt_v1, sampling_params=sampling_params)
print(o[0].outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [19:35<00:00, 1175.49s/it, est. speed input: 0.15 toks/s, output: 27.72 toks/s]

 To solve the problem, we need to find the coordinates of the point \( C \) on the segment \( \overline{AB} \) that does not belong to any other segment in the family \( \mathcal{F} \) except \( \overline{AB} \) itself. The family \( \mathcal{F} \) consists of unit-length segments \( \overline{PQ} \) with \( P \) on the x-axis and \( Q \) on the y-axis in the first quadrant.

First, we parameterize the points \( P \) and \( Q \). Let \( P = (a, 0) \) and \( Q = (0, b) \) where \( a, b > 0 \). Since the segment \( \overline{PQ} \) has unit length, we have:
\[
\sqrt{a^2 + b^2} = 1 \implies a^2 + b^2 = 1
\]

The segment \( \overline{PQ} \) can be parameterized by the line segment from \( (a, 0) \) to \( (0, b) \). Any point \( (x, y) \) on this segment can be written as:
\[
(x, y) = (a(1 - t), bt) \quad \text{for} \quad t \in [0, 1]
\]

We need to find a point \( C \) on \( \overline{AB} \) such that \( C \) does not lie on any other segment in \( \mathcal{F} \). The segment \( \overline{

Find one example where the original answer is right but the model fails on a modified one.

In [11]:
import importlib
import data  
import re

importlib.reload(data)

data_generator = data.data_generator

generator = data_generator('Maxwell-Jia/AIME_2024')  
prompt, answer = generator.generate_answer_2()

prompt = "<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n" + prompt + "<|im_end|>\n<|im_start|>assistant\n"
o = model.generate(prompt, sampling_params=sampling_params)
print(o[0].outputs[0].text)
print(f'answer: {answer}')
pattern = r"\\boxed\{(\d+)\}"
matches = re.findall(pattern, o[0].outputs[0].text)
print(matches)




\frac{1}{6}, \frac{\sqrt{35}}{6}


Processed prompts: 100%|██████████| 1/1 [15:56<00:00, 956.44s/it, est. speed input: 0.22 toks/s, output: 28.29 toks/s]

think
Okay, so I need to solve this problem where we have points O(0,0), A(1/6, 0), and B(0, √35/6). Then there's this family F of unit-length segments with P on the x-axis and Q on the y-axis in the first quadrant. We need to find a unique point C on AB (different from A and B) that isn't on any segment in F except AB. Then compute OC2 and express it as p/q where p and q are coprime, finally find p + q.

First, let me try to understand the problem. We have OA along the x-axis at (1/6, 0), and OB along the y-axis at (0, √35/6). Then AB is the line segment connecting A to B in the first quadrant. Then we are considering all unit-length segments with one endpoint on the x-axis (P) and the other on the y-axis (Q). So, these are segments of length 1 starting on the x-axis and ending on the y-axis. The point C is somewhere on AB, not at A or B, such that it doesn't lie on any other segment in F except AB. So, we need to show that there's a unique such point C, and then find the square of th